# Lesson 15: 버블차트 애니메이션

최규빈  
2023-07-26

<a href="https://colab.research.google.com/github/guebin/PP2024WIN/blob/main/posts/Day3/ls15.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# Import

In [1]:
import pandas as pd
import plotly.express as px

# `px.scatter_geo`

## 세계지도 그리기

`-` 기본그리기1

In [2]:
px.scatter_geo()

`-` 기본그리기2

In [3]:
px.scatter_geo(projection='natural earth')

## 세계지도 + 버블

`-` 좌표에 점을 찍어보기1

In [4]:
df=pd.DataFrame({'lat':[0,38],'lon':[0,127],'size':[5,20]})
df

In [5]:
px.scatter_geo(df,lat='lat',lon='lon',size='size')

`-` 좌표에 점을 찍어보기2

In [6]:
df=pd.DataFrame({'iso_alpha':['KOR','JPN'],'size':[10,2]})
df

In [7]:
px.scatter_geo(df,locations='iso_alpha',size='size')

## Gapminder data 시각화

`-` [Gapminder](https://en.wikipedia.org/wiki/Gapminder_Foundation)

-   스웨덴 스톡홀름에 등록된 비영리 벤처 기업
-   [Trendalyzer](https://en.wikipedia.org/wiki/Trendalyzer)
    소프트웨어를 개발하여 애니메이션 거품 그래프를 생성 $\to$
    소프트웨어는 2006년, Gapminder의 개발자는 2007년에 글고..

> <https://www.youtube.com/watch?v=N91kkT-vC6w>

`-` Gapminder data: 국가별 기대수명, 1인당 GDP, 인구에 대한 데이터 -
특징: 연도별로 정리가 되어있다.

In [8]:
df = px.data.gapminder()
df

`-` 2007년만 추출

In [9]:
df.query('year==2007')

`-` 시각화예시1: (x,y) = (lon,lat) = locations

In [10]:
px.scatter_geo(
    data_frame = df.query('year==2007'),
    locations= 'iso_alpha'
)

`-` 시각화예시2: (x,y) = (lon,lat) = locations, size= pop

In [11]:
px.scatter_geo(
    data_frame = df.query('year==2007'),
    locations= 'iso_alpha',
    size='pop'
)

`-` 시각화예시3: (x,y) = (lon,lat) = locations, size= pop, color=
continent

In [12]:
px.scatter_geo(
    data_frame = df.query('year==2007'),
    locations= 'iso_alpha',
    size='pop',
    color='continent'
)

# `px.scatter_geo` + animation

`-` 시각화예시1

In [13]:
px.scatter_geo(
    data_frame = df,
    locations= 'iso_alpha',
    size='pop',
    color='continent',
    animation_frame='year'
)

In [14]:
px.scatter_geo(
    data_frame = df,
    projection = 'natural earth',
    locations= 'iso_alpha',
    size='pop',
    color='continent',
    animation_frame='year'
)